In [1]:
%matplotlib inline

import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'src/')))
from sgpi.util import get_dataset, get_xy, get_r_squared, split, to_data_matrix, get_Phi

from sgpi import model
from sgpi.learner import SGRegressionLearner
import sqlalchemy as sa
from sqlalchemy import orm, func, or_, and_

from sgpi.plot import *
from scipy.linalg import svd

import pandas as pd
import pysgpp as sg; sg.omp_set_num_threads(4)

In [4]:
def duplicate_estimator(db_res):
    learner = SGRegressionLearner(db_res.grid_config, db_res.regularization_config,
                               db_res.solver_config, db_res.final_solver_config,
                               db_res.adaptivity_config)
    df = get_dataset(db_res.experiment.dataset)
    df, _ = split(df)
    X, y = get_xy(df)
    return X, y, learner

In [9]:
def get_df_lasso(weights, grid, X):
    active_set = weights.nonzero()[0]
    Phi = get_Phi(grid, X)
    active_Phi = Phi[:,active_set]
    rank = np.linalg.matrix_rank(active_Phi)
    return rank + 1

def get_df_ridge(Phi, lambda_reg):
    _, d, _ = svd(Phi)
    df = 0.0
    for sv in d:
        df = df + (sv**2)/(sv**2+lambda_reg)
    return df

def get_AIC(train_mse, num_data, df):
    return 2*df + num_data * np.log(train_mse)

In [10]:
ridge_t = sg.RegularizationType_Identity
diag_t = sg.RegularizationType_Diagonal
best_ridge = ses.query(model.Result, func.min_(model.Result.validation_mse)).\
    join(model.Experiment, model.RegularizationConfig, model.GridConfig).\
    filter(model.Experiment.dataset == 'concrete').\
    filter(or_(model.RegularizationConfig.type==ridge_t,\
               and_(model.RegularizationConfig.type==diag_t,model.RegularizationConfig.exponent_base==1.0 ))).\
    filter(model.GridConfig.T == 0.0).\
    filter(model.GridConfig.level >= 4).\
    group_by(model.GridConfig.level).\
    order_by(model.Result.validation_mse)

best_ridge.count()

2

In [11]:
for res, err in best_ridge:
    res.regularization_config.l1_ratio = 0.0
    X, y, estimator = duplicate_estimator(res)
    estimator.fit(X,y)
    grid = estimator._learner.getGrid()
    Phi = get_Phi(grid,X)
    mse = -estimator.score(X,y)
    df = get_df_ridge(Phi, res.regularization_config.lambda_reg)
    aic = get_AIC(mse, X.shape[0], df)
    print df, aic, res.train_mse, res.validation_mse

ValueError: object arrays are not supported